In [304]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# pymc
import pymc as pm
import pytensor
import pytensor.tensor as pt
#import pymc.sampling_jax
# import numpyro
# import blackjax
# import jax
# import arviz as az

# sklearn
from sklearn import preprocessing
from sklearn import metrics
from sklearn import linear_model
from sklearn.linear_model import Lasso

#statsmodels
import statsmodels.graphics.tsaplots as smg
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from statsmodels.tsa.stattools import kpss
import statsmodels.api as sm
import statsmodels.formula.api as smf


# warnings
import warnings
warnings.filterwarnings("ignore")

# plots
settings = {'figure.figsize':(14,4),
            'figure.dpi':144,
            'figure.facecolor':'w',
            'axes.spines.top':False,
            'axes.spines.bottom':False,
            'axes.spines.left':False,
            'axes.spines.right':False,
            'axes.grid':True,
            'grid.linestyle':'--',
            'grid.linewidth':0.5,
            'figure.constrained_layout.use':True}
plt.rcParams.update(settings)

In [305]:
def readdata():
    data = pd.read_csv(r'../data/hugo_data.csv', index_col=[0], date_format="mm/dd/yyyy")
    return data

raw_data = readdata()
print(raw_data)
print(raw_data.info())

           Start-ups      GDP         U     NAIRU  Job_vacancy        HICP  \
1/1/2006         NaN  179.989  6.433333  4.869898          NaN   86.326667   
4/1/2006         NaN  182.917  6.133333  4.877819          NaN   87.216667   
7/1/2006         NaN  183.853  5.933333  4.885739          NaN   87.123333   
10/1/2006        NaN  185.449  5.866667  4.893660          NaN   87.276667   
1/1/2007     46865.0  187.544  5.633333  4.936292          NaN   87.610000   
...              ...      ...       ...       ...          ...         ...   
4/1/2023     66970.0  234.334  3.466667  4.036260          5.0  128.120000   
7/1/2023     64570.0  233.457  3.633333  3.957463          4.8  129.320000   
10/1/2023    61245.0  234.031  3.566667  3.878666          4.7  128.356667   
1/1/2024     57750.0  233.302  3.633333  3.799049          4.6  129.153333   
4/1/2024     60065.0  235.674  3.633333  3.719431          4.6  131.796667   

           Cao-wages per hour  Borrowing_costs  Credit_standard

In [306]:
data = raw_data[['Start-ups', 
          'GDP', 
          'Job_vacancy',
          'Loans',
          'Zelfstandigenaftrek',
          'Cao-wages per hour',
          'U',
          'NAIRU'
          ]]

In [307]:
def initialregression(data1):

    data1.dropna(inplace=True)
    data1 = np.log(data1)
    data1 = data1.diff(-4)

    data1.dropna(inplace=True)

    y = data1.pop('Start-ups')
    X = data1
    X = sm.add_constant(X)

    model = sm.OLS(y,X)
    results = model.fit()
    print(results.summary())

data1 = data.copy()
initialregression(data1)

                            OLS Regression Results                            
Dep. Variable:              Start-ups   R-squared:                       0.208
Model:                            OLS   Adj. R-squared:                  0.106
Method:                 Least Squares   F-statistic:                     2.031
Date:                Tue, 15 Oct 2024   Prob (F-statistic):             0.0677
Time:                        18:46:23   Log-Likelihood:                 45.582
No. Observations:                  62   AIC:                            -75.16
Df Residuals:                      54   BIC:                            -58.15
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.0638    

In [308]:
# # one at a time
# data.dropna(inplace=True)

# data['Start_ups_percent'] = 100*((data['Start-ups']/data['Start-ups'].shift(4)) - 1)
# data['GDP_percent_lag']  = 100*((data['GDP'].shift(1)/data['GDP'].shift(5)) - 1)
# data['Job_vacancy_lag']  = data['Job_vacancy'].shift(2) - data['Job_vacancy'].shift(6)
# data['Loans']  = data['Loans'].shift(4)
# data['Zelfstandigenaftrek']  = 100*((data['Zelfstandigenaftrek']/data['Zelfstandigenaftrek'].shift(4)) - 1)

# data3 = data.dropna()
# data3 = data3.iloc[25:,]

# results = smf.ols("Start_ups_percent ~ 1 + GDP_percent_lag + Job_vacancy_lag + Loans + Zelfstandigenaftrek", data=data3).fit()

# # Inspect the results
# print(results.summary())

In [309]:
data

,Start-ups,GDP,Job_vacancy,Loans,Zelfstandigenaftrek,Cao-wages per hour,U,NAIRU
1/1/2006,NaN,179.989,NaN,185.370333,6500,77.538897,6.433333,4.869898
4/1/2006,NaN,182.917,NaN,191.778333,6500,77.850071,6.133333,4.877819
7/1/2006,NaN,183.853,NaN,193.400333,6500,78.161245,5.933333,4.885739
10/1/2006,NaN,185.449,NaN,207.998667,6500,78.161245,5.866667,4.893660
1/1/2007,46865.0,187.544,NaN,230.103000,6500,78.868458,5.633333,4.936292
...,...,...,...,...,...,...,...,...
4/1/2023,66970.0,234.334,5.0,354.648667,5030,110.833333,3.466667,4.036260
7/1/2023,64570.0,233.457,4.8,355.830667,5030,112.666667,3.633333,3.957463
10/1/2023,61245.0,234.031,4.7,364.335333,5030,113.633333,3.566667,3.878666
1/1/2024,57750.0,233.302,4.6,365.155667,3750,117.233333,3.633333,3.799049


In [310]:
# one at a time
#data.dropna(inplace=True)

data['Start_ups_percent'] = 100*(np.log(data['Start-ups']).diff(4))
data['GDP_percent_lag']  = 100*((np.log(data['GDP'])).diff(4)).shift(1)
data['Job_vacancy_lag']  = data['Job_vacancy'].diff(4).shift(2)
#data['Loans_lag']  = data['Loans'].shift(4)
data['U_NAIRU'] = data['U'].shift(4) - data['NAIRU'].shift(4)
data['Zelfstandigenaftrek_lag']  = 100*(np.log(data['Zelfstandigenaftrek']).diff(4)).shift(4)
data['Cao_wages_per_hour_lag'] = 100*(np.log(data['Cao-wages per hour']).diff(4)).shift(3)
data.head(10)


,Start-ups,GDP,Job_vacancy,Loans,Zelfstandigenaftrek,Cao-wages per hour,U,NAIRU,Start_ups_percent,GDP_percent_lag,Job_vacancy_lag,U_NAIRU,Zelfstandigenaftrek_lag,Cao_wages_per_hour_lag
1/1/2006,NaN,179.989,NaN,185.370333,6500,77.538897,6.433333,4.869898,NaN,NaN,NaN,NaN,NaN,NaN
4/1/2006,NaN,182.917,NaN,191.778333,6500,77.850071,6.133333,4.877819,NaN,NaN,NaN,NaN,NaN,NaN
7/1/2006,NaN,183.853,NaN,193.400333,6500,78.161245,5.933333,4.885739,NaN,NaN,NaN,NaN,NaN,NaN
10/1/2006,NaN,185.449,NaN,207.998667,6500,78.161245,5.866667,4.893660,NaN,NaN,NaN,NaN,NaN,NaN
1/1/2007,46865.0,187.544,NaN,230.103000,6500,78.868458,5.633333,4.936292,NaN,NaN,NaN,1.563435,NaN,NaN
4/1/2007,50110.0,188.697,NaN,236.345000,6500,79.264498,5.266667,4.978923,NaN,4.111775,NaN,1.255514,NaN,NaN
7/1/2007,43325.0,190.776,NaN,244.364333,6500,79.915134,5.233333,5.021555,NaN,3.111006,NaN,1.047594,NaN,NaN
10/1/2007,35945.0,193.551,NaN,256.056000,6500,80.028289,5.066667,5.064187,NaN,3.696344,NaN,0.973007,NaN,1.700168
1/1/2008,33460.0,194.907,3.4,274.882667,6500,81.188119,4.800000,5.094126,-33.692043,4.276113,NaN,0.697042,0.0,1.800553
4/1/2008,47585.0,195.047,3.4,290.752000,6500,81.951909,4.800000,5.124065,-5.170300,3.850904,NaN,0.287743,0.0,2.219132


In [311]:

data3 = data.dropna()

data3 = data3.iloc[20:,]

results = smf.ols("Start_ups_percent ~ 1 + GDP_percent_lag + Job_vacancy_lag  + U_NAIRU + Cao_wages_per_hour_lag", data=data3).fit(cov_type='HAC',cov_kwds={'maxlags':5})

# Inspect the results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:      Start_ups_percent   R-squared:                       0.685
Model:                            OLS   Adj. R-squared:                  0.650
Method:                 Least Squares   F-statistic:                     56.15
Date:                Tue, 15 Oct 2024   Prob (F-statistic):           9.51e-15
Time:                        18:46:23   Log-Likelihood:                -115.21
No. Observations:                  40   AIC:                             240.4
Df Residuals:                      35   BIC:                             248.9
Df Model:                           4                                         
Covariance Type:                  HAC                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 12